# Fire Spot Features
Source: [NASA FIRMS API](https://firms.modaps.eosdis.nasa.gov/api/)

## Imports

In [2]:
import numpy as np
import pandas as pd
import math
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from io import StringIO
import datetime as dt
from pathlib import Path

## API Connection


Connecting the database on real time fire data from NASA FIRMS.

Instructions: input information below based on the parameters from the last active session on the FIRMS API.

Access to the API key: [API Key](https://https://firms.modaps.eosdis.nasa.gov/api/)


In [ ]:
MAP_KEY = "[KEY]"
SOURCE = "VIIRS_NOAA20_NRT"
COUNTRY_ID = "USA"
DAY_RANGE = 10
START_DATE = "2025-01-05"

base_url = "https://firms.modaps.eosdis.nasa.gov/api/country/csv"
url = f"{base_url}/{MAP_KEY}/{SOURCE}/{COUNTRY_ID}/{DAY_RANGE}/{START_DATE}"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Print the first few lines of the CSV data
    # Adjust the number to display more or fewer characters

    try:
        # Parse the CSV data into a Pandas DataFrame
        firms_df = pd.read_csv(StringIO(response.text))

        # Print the DataFrame
        print(firms_df.head())
    except pd.errors.ParserError as e:
        print(f"ParserError: {e}")
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code}")

  country_id  latitude  longitude  bright_ti4  scan  track    acq_date  \
0        USA  19.39648 -155.28795      328.52  0.53    0.5  2025-01-05   
1        USA  19.39706 -155.28281      329.05  0.53    0.5  2025-01-05   
2        USA  19.40052 -155.29370      337.20  0.53    0.5  2025-01-05   
3        USA  19.40112 -155.28838      354.98  0.53    0.5  2025-01-05   
4        USA  19.40170 -155.28322      356.35  0.53    0.5  2025-01-05   

   acq_time satellite instrument confidence version  bright_ti5    frp  \
0         2       N20      VIIRS          n  2.0NRT      294.68   5.03   
1         2       N20      VIIRS          n  2.0NRT      297.70   5.03   
2         2       N20      VIIRS          n  2.0NRT      302.34  20.00   
3         2       N20      VIIRS          n  2.0NRT      313.49  32.27   
4         2       N20      VIIRS          l  2.0NRT      297.77  32.27   

  daynight  
0        D  
1        D  
2        D  
3        D  
4        D  


## Datetime Conversion

In [ ]:
# Convert acq_time to a 4-digit string (e.g., '841' -> '0841')
firms_df['acq_time'] = firms_df['acq_time'].astype(str).str.zfill(4)

# Create a new datetime column by combining acq_date and acq_time
firms_df['datetime'] = pd.to_datetime(
    firms_df['acq_date'] + ' ' + 
    firms_df['acq_time'].str.slice(0, 2) + ':' + 
    firms_df['acq_time'].str.slice(2, 4),
    format="%Y-%m-%d %H:%M"
)

## Coordinate Filtering
Create bounding box for north, south, west and east directions. Examples below

Find bounding box coordinates: [bboxfinder](http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000)

In [ ]:
# LA Bounding Box
firms_df_la = firms_df[
    (firms_df['latitude'] >= 32) &
    (firms_df['latitude'] <= 36) &
    (firms_df['longitude'] >= -120) &
    (firms_df['longitude'] <= -116)
]

# California Bounding Box 
firms_df_ca = firms_df[
    (firms_df['latitude'] >= 32.5343) &
    (firms_df['latitude'] <= 42.0095) &
    (firms_df['longitude'] >= -124.4096) &
    (firms_df['longitude'] <= -114.1315)
]

### Save Dataset

In [ ]:
# As .csv file
output_path = Path("../fire_data/firms_df_ca.csv")
# Ensure that the parent directories exist
output_path.parent.mkdir(parents=True, exist_ok=True)
firms_df_ca.to_csv(output_path, index=False)